<a href="https://colab.research.google.com/github/Gedeon-m-gedus/Image_Processing/blob/master/CV_10_Detectron2_custom_pascal_voc_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training object detection model on my custom dataset

## 1. DTECTRON2

In [ ]:
# Downloading data
from google.colab import drive
drive.mount('/content/gdrive')#
#root_path = 'gdrive/My Drive/AMMI_project/object_detector'

In [ ]:
#Downloading data from my google drive
!unzip -q /content/gdrive/My\ Drive/AMMI_project/object_detector/data.zip -d dataset/
#Downloading data from my google drive
#!unzip -q /content/gdrive/My\ Drive/AMMI_project/object_detector/data2.zip -d dataset2/

### Imports

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html >/dev/null
!pip install cython pyyaml==5.1 >/dev/null
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' >/dev/null
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html >/dev/null

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#from detectron2.data.datasets import register_pascal_voc



In [ ]:
name = 'mydata'
dirname = './dataset/data/'
dirname2 = './dataset2/data2/'

In [ ]:
import numpy as np
import os
import xml.etree.ElementTree as ET
from typing import List, Tuple, Union
from fvcore.common.file_io import PathManager

from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode

In [ ]:

__all__ = ["load_voc_instances", "register_pascal_voc"]


# fmt: off
CLASS_NAMES = ('giraffe',
                'person',
                'zebra',
                'elephant',
                'impala',
                'monkey',
                'lion',
                'leopard',
                'crocodile',
                'buffalo',
                'hyna',
                'bird',
                'gorilla')
# fmt: on

def load_voc_instances(dirname: str, split: str, class_names: Union[List[str], Tuple[str, ...]]):
    """
    Load Pascal VOC detection annotations to Detectron2 format.
    Args:
        dirname: Contain "Annotations", "ImageSets", "JPEGImages"
        split (str): one of "train", "test", "val", "trainval"
        class_names: list or tuple of class names
    """
    with PathManager.open(os.path.join(dirname, split + ".txt")) as f:
        fileids = np.loadtxt(f, dtype=np.str)

    # Needs to read many small annotation files. Makes sense at local
    annotation_dirname = PathManager.get_local_path(os.path.join(dirname, "annotations/"))
    dicts = []
    for fileid in fileids:
        anno_file = os.path.join(annotation_dirname, fileid + ".xml")
        jpeg_file = os.path.join(dirname, "images/", fileid + ".jpg")

        with PathManager.open(anno_file) as f:
            tree = ET.parse(f)

        r = {
            "file_name": jpeg_file,
            "image_id": fileid,
            "height": int(tree.findall("./size/height")[0].text),
            "width": int(tree.findall("./size/width")[0].text),
        }
        instances = []

        for obj in tree.findall("object"):
            cls = obj.find("name").text
            # We include "difficult" samples in training.
            # Based on limited experiments, they don't hurt accuracy.
            # difficult = int(obj.find("difficult").text)
            # if difficult == 1:
            # continue
            bbox = obj.find("bndbox")
            bbox = [float(bbox.find(x).text) for x in ["xmin", "ymin", "xmax", "ymax"]]
            # Original annotations are integers in the range [1, W or H]
            # Assuming they mean 1-based pixel indices (inclusive),
            # a box with annotation (xmin=1, xmax=W) covers the whole image.
            # In coordinate space this is represented by (xmin=0, xmax=W)
            bbox[0] -= 1.0
            bbox[1] -= 1.0
            instances.append(
                {"category_id": class_names.index(cls), "bbox": bbox, "bbox_mode": BoxMode.XYXY_ABS}
            )
        r["annotations"] = instances
        dicts.append(r)
    return dicts


def register_pascal_voc(name, dirname, split, year, class_names=CLASS_NAMES):
    meta_data = DatasetCatalog.register(name, lambda: load_voc_instances(dirname, split, class_names))
    catalog = MetadataCatalog.get(name).set(
        thing_classes=list(class_names), dirname=dirname, year=year, split=split
    )
    #return meta_data, catalog

In [ ]:
#train_meta_data, train_catalog = 
register_pascal_voc('train_data', dirname, split = 'train', year='2020', class_names=CLASS_NAMES)

In [ ]:
train_dataset_dicts = DatasetCatalog.get('train_data')
train_metadata=MetadataCatalog.get('train_data')

In [ ]:
register_pascal_voc('valid_data', dirname, split = 'valid', year='2020', class_names=CLASS_NAMES)

In [ ]:
valid_dataset_dicts = DatasetCatalog.get('valid_data')
valid_metadata=MetadataCatalog.get('valid_data')

In [ ]:
valid_dataset_dicts[0]

In [ ]:
images_index = [0,7,78,156,444]
for index in images_index:
    img = cv2.imread(train_dataset_dicts[index]["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(train_dataset_dicts[index])
    print('visualizing ',train_dataset_dicts[index]["file_name"])
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
def rcnn_R_50(init_weights_flag = True, path = '/COCO_init_model'):


      """
      This function takes two parameters:
        1.  init_weights_flag : This is a flag that defines the initial weights of the backbone network of our R-CNN
        2. path : this is the path where I need to save my model
      
      This function returns the predictor, after training the model it will loads the weights from the directory and the model configurations(cfg)

      Parameters are defined in the same way as our lecturer said:
        train for 300 iterations, a start learning rate of 0.02, 2 images per batch, and 128 regions per batch
      """


      cfg = get_cfg()
      cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
      cfg.DATASETS.TRAIN = ("train_data",)
      cfg.DATASETS.TEST = ()
      cfg.DATALOADER.NUM_WORKERS = 2
      cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
      cfg.SOLVER.IMS_PER_BATCH = 8 # images per batch
      cfg.SOLVER.BASE_LR = 0.02  # Learning rate
      cfg.SOLVER.MAX_ITER = 300    # number of iterations 
      cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 #regions per batch
      cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13  #three classes
      cfg.with_coco_init = init_weights_flag # Flag to initialize weight to COCO or IMAGENET

      #create a path where I will store my model, this path defers for each model
      os.makedirs(path, exist_ok=True)
      cfg.OUTPUT_DIR = path
      trainer = DefaultTrainer(cfg) 
      trainer.resume_or_load(resume=False)
      trainer.train()

      ## After training lets return the predictor
      cfg.MODEL.WEIGHTS = os.path.join(path, "model_final.pth")
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set the testing threshold for this model
      cfg.DATASETS.TEST = ("valid_data", )

      return DefaultPredictor(cfg),cfg,trainer

In [ ]:
coco_init_predictor, coco_init_cfg, coco_init_trainer = rcnn_R_50()

In [ ]:
# # coco_init_predictor.input_format
# %load_ext tensorboard
# %tensorboard --logdir COCO_init_model

In [ ]:
images_index = [8,16,44]
for index in images_index:
    img = cv2.imread(train_dataset_dicts[index]["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(train_dataset_dicts[index])
    print('visualizing ',train_dataset_dicts[index]["file_name"])
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
#train_dataset_dicts[9]

In [ ]:
from google.colab.patches import cv2_imshow
for d in valid_dataset_dicts:
  img = cv2.imread(d["file_name"])
  prediction = coco_init_predictor(img)
  visualizer = Visualizer(img[:, :, ::-1],
                                metadata=valid_metadata,
                                scale=0.5)
  vis = visualizer.draw_instance_predictions(prediction["instances"].to("cpu"))
  cv2_imshow( vis.get_image()[:, :, ::-1])

In [ ]:
coco_init_predictor, coco_init_cfg, coco_init_trainer = rcnn_R_50(init_weights_flag = True,)